In [10]:
import pandas as pd
from matplotlib import pyplot as plt
import glob
import matplotlib.dates as mdates
import numpy as np
from datetime import timedelta
from tqdm import tqdm
import datetime as dt
from datetime import time
import math
from datetime import datetime

In [11]:
def reader(path, first_var):
    '''
    Reads SEMS/DASH data, adds datetime columns

    :param path: path to data file
    :param first_var: the name of the fir column label
    :return: pandas DataFrame
    '''
    # Open the file and read the lines
    skip=1
    with open(path, "r") as file:
        # Iterate over the lines
        for line in file:
            # rip leading and trailing whitespace
            line = line.strip()
            # Check if the line contains column names
            if line.startswith(first_var):
                # Split the line by whitespace and append to the columns list
                columns = line[1:].strip().split("\t")
                break
            skip+=1
    # Read the data into a DataFrame, skipping the first 6 rows of comments
    d = pd.read_csv(path, sep='\t', skiprows=skip, names=columns, low_memory=False)
    #Check for duplicated metadata, remove metadata rows based on string "OPC SN"
    if len(d)>0:
        if isinstance(d.iloc[0,0], str):
            dup_meta = [n for n, i in enumerate(d.iloc[:,0]) if 'OPC SN' in i]
            if len(dup_meta) > 0:
                for line in dup_meta:
                    #Deletes duplicate metadata rows from dataframe
                    d.drop(np.arange(line,line+68), inplace = True)
                    # Apply the function to each column
                    d_og = d.copy()
                    for c in d.keys():
                        try:
                            d[c] = pd.to_numeric(d_og[c])
                        except:
                            d[c] = d_og[c]
                    #d = d.apply(pd.to_numeric, ‘raise’,)
                    
    # Creates datetime columns
    if 'DOY.Frac' in d.keys():
        d['dt'] = pd.to_datetime('2024-1-1') + pd.to_timedelta(d['DOY.Frac'], unit='D') - pd.Timedelta(days=1)
    if 'StartTimeSt' in d.keys():
        d['st_dt'] = pd.to_datetime('2024-1-1') + pd.to_timedelta(d['StartTimeSt'], unit='D') - pd.Timedelta(days=1)
    if 'EndTimeSt' in d.keys():
        d['end_dt'] = pd.to_datetime('2024-1-1') + pd.to_timedelta(d['EndTimeSt'], unit='D') - pd.Timedelta(days=1)
    if 'YY/MM/DD' and 'HR:MN:SC' in d.keys():
        d['dt'] = pd.to_datetime(str(20) + d['YY/MM/DD'] + ' ' + d['HR:MN:SC'], format='%Y/%m/%d %H:%M:%S')
    return d

In [12]:
def glob_reader(file_key, first_var, subfolder = './data/'):
    '''
    Reads groups of data files and merges them into one

    :param file_key: shared key in filenames
    :param first_var: the name of the first column label
    :param subfolder: name of the subfolder containing the data
    :return: pandas DataFrame
    '''
    paths = sorted(glob.glob(subfolder+'*'+file_key+'*'))
    d = []
    for i in range(0, len(paths)):
        f = reader(paths[i], first_var)
        if len(f)>0:
            d.append(f)
    d = pd.concat(d).reset_index()
    return d

In [13]:
def read_all_data():

    folders = sorted(glob.glob('./data/DASH-flight*'))

    dopc = []
    hopc = []
    dash = []
    sems = []

    for i in range(0, len(folders)):
        path = folders[i] + '/'
        dopc.append(glob_reader('OPC_212', '#YY/MM/DD', subfolder = path).drop_duplicates(subset='dt', keep='first'))
        hopc.append(glob_reader('OPC_210', '#YY/MM/DD', subfolder = path).drop_duplicates(subset='dt', keep='first'))
        dash.append(glob_reader('DASH_FLOW', '#DOY.Frac', subfolder = path).drop_duplicates(subset='dt', keep='first'))
        sems.append(glob_reader('SEMS_DATA', '#DOY.Frac', subfolder = path).drop_duplicates(subset='dt', keep='first'))
    
    dopc = pd.concat(dopc, ignore_index=True)
    hopc = pd.concat(hopc, ignore_index=True)
    dash = pd.concat(dash, ignore_index=True)
    sems = pd.concat(sems, ignore_index=True)
    
    # merge the DataFrames based on the DOPC times
    merged = pd.merge_asof(dopc, hopc, on='dt', direction = 'nearest', tolerance=timedelta(seconds=.9)).drop(columns=['index_x']).drop_duplicates(subset='dt', keep='first').reset_index()
    merged = pd.merge_asof(merged, dash, on='dt', direction = 'nearest', tolerance=timedelta(seconds=.9)).drop(columns=['index_x']).drop_duplicates(subset='dt', keep='first').reset_index()
    merged = pd.merge_asof(merged, sems, on='dt', direction = 'nearest', tolerance=timedelta(seconds=.9)).drop(columns=['index_x']).drop_duplicates(subset='dt', keep='first').reset_index()

    return merged

In [14]:
ft = pd.read_csv('./meta/ARCSIX_takeoff_landing_times.txt', parse_dates=['Takeoff_UTC', 'Landing_UTC', 'LARGE_Filter_Off_UTC', 'LARGE_Filter_On_UTC'])
ls = pd.read_csv('./meta/ARCSIX_DASH_SEMS_switch_times.txt', parse_dates=['Switch_Start_UTC', 'Switch_Stop_UTC'])
d = read_all_data()

In [15]:
ls

,FltNum,Switch_Start_UTC,Switch_Stop_UTC,LARGE_From,LARGE_To,Notes
0,1,2024-05-28 12:01:00,NaT,SEMS,OFF,"For RF1, we removed SEMS-DASH completely from ..."
1,1,NaT,2024-05-28 15:23:00,OFF,SEMS,This means that LARGE manifold was connected t...
2,2,2024-05-30 11:07:30,2024-05-30 11:08:30,SEMS,DASH,Switch from SEMS-DASH to DASH Only; Extended t...
3,2,2024-05-30 13:25:00,2024-05-30 13:26:00,DASH,SEMS,Switch from DASH Only to SEMS-DASH; Extended t...
4,2,2024-05-30 16:28:00,2024-05-30 16:29:00,SEMS,DASH,NaN
5,2,2024-05-30 17:47:00,2024-05-30 17:48:55,DASH,SEMS,NaN
6,3,2024-05-31 12:37:25,2024-05-31 12:38:40,SEMS,DASH,NaN
7,3,2024-05-31 15:20:30,2024-05-31 15:21:30,DASH,SEMS,NaN
8,3,2024-05-31 16:47:10,2024-05-31 16:48:30,SEMS,DASH,NaN
9,3,2024-05-31 18:21:00,2024-05-31 18:22:10,DASH,SEMS,NaN


In [16]:
for i in range(0, len(ls)):
    if pd.isna(ls.loc[i,'Switch_Start_UTC']):
        ls.loc[i,'Switch_Start_UTC'] = ls.loc[i,'Switch_Stop_UTC'] - timedelta(seconds=10)
    if pd.isna(ls.loc[i,'Switch_Stop_UTC']):
        ls.loc[i,'Switch_Stop_UTC'] = ls.loc[i,'Switch_Start_UTC'] + timedelta(seconds=10)

In [17]:
ls

,FltNum,Switch_Start_UTC,Switch_Stop_UTC,LARGE_From,LARGE_To,Notes
0,1,2024-05-28 12:01:00,2024-05-28 12:01:10,SEMS,OFF,"For RF1, we removed SEMS-DASH completely from ..."
1,1,2024-05-28 15:22:50,2024-05-28 15:23:00,OFF,SEMS,This means that LARGE manifold was connected t...
2,2,2024-05-30 11:07:30,2024-05-30 11:08:30,SEMS,DASH,Switch from SEMS-DASH to DASH Only; Extended t...
3,2,2024-05-30 13:25:00,2024-05-30 13:26:00,DASH,SEMS,Switch from DASH Only to SEMS-DASH; Extended t...
4,2,2024-05-30 16:28:00,2024-05-30 16:29:00,SEMS,DASH,NaN
5,2,2024-05-30 17:47:00,2024-05-30 17:48:55,DASH,SEMS,NaN
6,3,2024-05-31 12:37:25,2024-05-31 12:38:40,SEMS,DASH,NaN
7,3,2024-05-31 15:20:30,2024-05-31 15:21:30,DASH,SEMS,NaN
8,3,2024-05-31 16:47:10,2024-05-31 16:48:30,SEMS,DASH,NaN
9,3,2024-05-31 18:21:00,2024-05-31 18:22:10,DASH,SEMS,NaN


In [18]:
def retrieve_flows(d):
    
    # filter out for flight times
    d_out = pd.DataFrame()
    f_ac = 0
    for _, row in ft.iterrows():
        mask = (d['dt'] >= row['LARGE_Filter_Off_UTC']) & (d['dt'] <= row['LARGE_Filter_On_UTC'])
        filtered = d.loc[mask]
        d_out = pd.concat([d_out, filtered], ignore_index=True)
    d_out['input_flow_lpm'] = d_out['UpSt_Samp']
    for i, row in ls.iterrows():
        if row['LARGE_To'] == 'OFF':
            mask = (d_out['dt'] >= row['Switch_Start_UTC']) & (d_out['dt'] <= row['Switch_Stop_UTC'])
            d_out.loc[mask,'input_flow_lpm'] = np.nan
            mask = (d_out['dt'] >= row['Switch_Stop_UTC']) & (d_out['dt'] <= ls.loc[i+1,'Switch_Start_UTC'])
            d_out.loc[mask,'input_flow_lpm'] = 0.0
        elif row['LARGE_To'] == 'DASH':
            mask = (d_out['dt'] >= row['Switch_Start_UTC']) & (d_out['dt'] <= row['Switch_Stop_UTC'])
            #print('DASH', len(mask[mask==True]), row['Switch_Start_UTC'], row['Switch_Stop_UTC'], np.nan)
            d_out.loc[mask,'input_flow_lpm'] = np.nan

            mask = (d_out['dt'] >= row['Switch_Start_UTC']+timedelta(seconds=10)) & (d_out['dt'] <= row['Switch_Stop_UTC']-timedelta(seconds=10))
            #print('DASH', len(mask[mask==True]), row['Switch_Start_UTC'], row['Switch_Stop_UTC'], np.nan)
            d_out.loc[mask,'input_flow_lpm'] = 0


            mask = (d_out['dt'] >= row['Switch_Stop_UTC']) & (d_out['dt'] <= ls.loc[i+1,'Switch_Stop_UTC'])
            #print('DASH2', len(mask[mask==True]), row['Switch_Stop_UTC'], ls.loc[i+1,'Switch_Start_UTC'], 'HM_Smp_Xs')
            d_out.loc[mask,'input_flow_lpm'] = d_out.loc[mask,'HM_Smp_Xs']
        elif row['LARGE_To'] == 'SEMS':
            mask = (d_out['dt'] >= row['Switch_Start_UTC']) & (d_out['dt'] <= row['Switch_Stop_UTC'])
            #print('SEMS', len(mask[mask==True]), row['Switch_Start_UTC'], row['Switch_Stop_UTC'], np.nan)
            d_out.loc[mask,'input_flow_lpm'] = np.nan

            mask = (d_out['dt'] >= row['Switch_Start_UTC']+timedelta(seconds=10)) & (d_out['dt'] <= row['Switch_Stop_UTC']-timedelta(seconds=10))
            #print('DASH', len(mask[mask==True]), row['Switch_Start_UTC'], row['Switch_Stop_UTC'], np.nan)
            d_out.loc[mask,'input_flow_lpm'] = 0
    
    output = pd.DataFrame(data={'datetime_UTC':d_out['dt'], 'input_flow_lpm':d_out['input_flow_lpm']})

    return output.replace(-0.0, 0.0).reset_index().drop(columns=['index'])

In [19]:
d = read_all_data()

In [20]:
d2 = retrieve_flows(d)

In [21]:
d2.to_csv('./tables/DASH_flow.csv', index=False)

In [38]:
d2[d2['input_flow_lpm']<0]

,datetime_UTC,input_flow_lpm
478,2024-05-28 12:00:28,-0.04
479,2024-05-28 12:00:29,-0.36
9974,2024-05-28 15:23:28,-0.78
9975,2024-05-28 15:23:29,-0.91
9976,2024-05-28 15:23:30,-0.36
...,...,...
264183,2024-06-17 18:10:30,-0.39
264184,2024-06-17 18:10:31,-0.30
264251,2024-06-17 18:11:38,-0.01
264403,2024-06-17 18:14:13,-0.13


In [39]:
d2[pd.isna(d2['input_flow_lpm'])]

,datetime_UTC,input_flow_lpm
510,2024-05-28 12:01:00,NaN
511,2024-05-28 12:01:01,NaN
512,2024-05-28 12:01:02,NaN
513,2024-05-28 12:01:03,NaN
514,2024-05-28 12:01:04,NaN
...,...,...
122275,2024-06-06 11:24:56,NaN
122276,2024-06-06 11:24:57,NaN
122277,2024-06-06 11:24:58,NaN
122278,2024-06-06 11:24:59,NaN


In [40]:
d2['date'] = d2['datetime_UTC'].dt.date

In [50]:
d2[d2['input_flow_lpm']<0].groupby('date').count()/60

,datetime_UTC,input_flow_lpm
date,,
2024-05-28,0.133333,0.133333
2024-05-30,12.283333,12.283333
2024-05-31,0.500000,0.500000
2024-06-03,0.400000,0.400000
2024-06-05,3.566667,3.566667
2024-06-06,1.916667,1.916667
2024-06-07,11.416667,11.416667
2024-06-10,0.766667,0.766667
2024-06-11,3.000000,3.000000
